In [1]:
import ee
import geemap
import pandas as pd
import geemap.colormaps as cm

In [2]:
ee.Initialize()

In [3]:
#Import declare the shapefiles
Ndhiwa_shp = 'Ndhiwa.shp'
Plot_shp = 'plot.shp'
Plot = geemap.shp_to_ee(Plot_shp)
Ndhiwa = geemap.shp_to_ee(Ndhiwa_shp)

In [4]:
# Sugarcane planting period
# Sugar seasion = 20 months average

# start date
s_date = '2019-04-01' #change the date
# end date
e_date = '2021-01-30'

# modis NDVI collection/ filter by date
modis = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI')
modis = modis.filterDate(s_date, e_date)
#modis = modis.filterBounds(Ndhiwa)

print('Number of dataset available within the specified =',modis.size().getInfo())

Number of dataset available within the specified = 42


In [58]:
Map = geemap.Map(center=(0.9613,34.6536),zoom=8)
Map

Map(center=[0.9613, 34.6536], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [60]:
# Visualization color ramp
# palette = cm.palettes.ndvi
# ndvi_vis = {
#   'min': 0.0,
#   'max': 9000,
#   'palette': palette
# }

#Convert Imagecollection to bands
NDVI_bands = modis.toBands()

#Clip to AOI
Ndhiwa_NDVI = NDVI_bands.clip(Ndhiwa)

#Display on map
Map.addLayer(Plot, name='Plots')
Map.addLayer(Ndhiwa_NDVI, name='Ndhiwa NDVI')



In [41]:
#get the average NDVI per plot
plot_ndvi_modis = NDVI_bands.reduceRegions(Plot,reducer=ee.Reducer.mean(), scale =250).getInfo()

#subset the featurecollection
p_feature = plot_ndvi_modis['features']

#read as dataframe
dff = pd.DataFrame(p_feature)
dff.head()

,type,geometry,id,properties
0,Feature,"{'type': 'Polygon', 'coordinates': [[[34.33138...",0,"{'2019_04_07_NDVI': 6276.648648648649, '2019_0..."
1,Feature,"{'type': 'Polygon', 'coordinates': [[[34.32737...",1,"{'2019_04_07_NDVI': 6165.380952380952, '2019_0..."
2,Feature,"{'type': 'Polygon', 'coordinates': [[[34.31373...",2,"{'2019_04_07_NDVI': 6196, '2019_04_23_NDVI': 7..."
3,Feature,"{'type': 'Polygon', 'coordinates': [[[34.31958...",3,"{'2019_04_07_NDVI': 6117.999999999999, '2019_0..."
4,Feature,"{'type': 'Polygon', 'coordinates': [[[34.31926...",4,"{'2019_04_07_NDVI': 6652, '2019_04_23_NDVI': 7..."


In [42]:
#from the dataframe extract on the NDVI attribute and the plot names
plotnn = pd.Series(dff['properties'])
plotnn.head(10) #not yet suitable 

0    {'2019_04_07_NDVI': 6276.648648648649, '2019_0...
1    {'2019_04_07_NDVI': 6165.380952380952, '2019_0...
2    {'2019_04_07_NDVI': 6196, '2019_04_23_NDVI': 7...
3    {'2019_04_07_NDVI': 6117.999999999999, '2019_0...
4    {'2019_04_07_NDVI': 6652, '2019_04_23_NDVI': 7...
5    {'2019_04_07_NDVI': 6405, '2019_04_23_NDVI': 7...
6    {'2019_04_07_NDVI': 6257, '2019_04_23_NDVI': 7...
7    {'2019_04_07_NDVI': 6328.655172413793, '2019_0...
8    {'2019_04_07_NDVI': 6186.483516483517, '2019_0...
9    {'2019_04_07_NDVI': 6507.571428571428, '2019_0...
Name: properties, dtype: object

In [ ]:
#Concatenate the multiple series and read as dataframe
testin = pd.concat([pd.Series(d) for d in plotnn], axis = 1)
#https://stackoverflow.com/questions/48811096/creating-a-dataframe-from-multiple-dictionaries
testin #still not good enough

In [44]:
#Transpose to have each attribute on different columns
nte = testin.transpose() #good
nte

,2019_04_07_NDVI,2019_04_23_NDVI,2019_05_09_NDVI,2019_05_25_NDVI,2019_06_10_NDVI,2019_06_26_NDVI,2019_07_12_NDVI,2019_07_28_NDVI,2019_08_13_NDVI,2019_08_29_NDVI,...,2020_09_13_NDVI,2020_09_29_NDVI,2020_10_15_NDVI,2020_10_31_NDVI,2020_11_16_NDVI,2020_12_02_NDVI,2020_12_18_NDVI,2021_01_01_NDVI,2021_01_17_NDVI,Name
0,6276.648649,7578.135135,7593.216216,8140.162162,7764.837838,7261.405405,7204.189189,6799.702703,5735.351351,6752.621622,...,6419.027027,6877.837838,7382.540541,7526.027027,7410.459459,7991.189189,7715.351351,6944.972973,6329.189189,plot 1
1,6165.380952,6919.238095,7451.428571,7596,7472.857143,6937.571429,6927.809524,6479.857143,5750.666667,6776.47619,...,5847.952381,6720.428571,7101.380952,7290.666667,7367.666667,7762.619048,7332,6896.190476,6285.761905,plot 2
2,6196,7636.333333,7607.0,7872.833333,7231.0,6750.5,7087.166667,6226.5,5691.166667,6629.333333,...,6185.166667,7094.666667,7402.5,7599.166667,7449.166667,7718.166667,6624.0,6971.333333,6325.333333,plot 3
3,6118.0,7009,7432.043478,7691.304348,7510.652174,6806.304348,6676.043478,6313.782609,5845.608696,6631.391304,...,5840.782609,6792.826087,7100.043478,7360,7355.565217,7787.086957,6680.043478,7027.73913,6288,plot 5
4,6652,7596,7765,7947,5149,6845,6766,6466,5539,6741,...,5873,6607,7171,7528,7253,7844,6691,6846,6374,plot 6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,6722.044444,7583.177778,7510.777778,7760.2,7448.688889,7161.311111,7219.133333,6823.866667,6222.444444,6756.977778,...,6312.577778,6842.577778,7139.733333,7187.666667,7312.355556,7542.222222,7051.977778,6986.4,6820.133333,plot 67
71,6859,7558,7621,7814,7385,7200,7239,6949,6331,7115,...,6237,6544,7108,7330,7407,7620,7172,6995,6866,plot 68
72,6421.571429,7223,7368.714286,7804.714286,7308.285714,7119.285714,7097.142857,6856.285714,6298.857143,6732.714286,...,6376.142857,6853.714286,7047.571429,7318.285714,7276.714286,7513.428571,7135.0,6958.285714,6822.571429,plot 69
73,6496.909091,7072.181818,7966.545455,7625.545455,7380.636364,7182.727273,7132.0,6913.909091,6417.454545,6855.454545,...,6344.272727,6707.454545,7187.272727,7230.818182,7228.454545,7552.454545,7127.272727,7067.181818,7046.545455,plot 70


In [52]:
def image_array_to(arr, list_of_bands):
    #Bands dict to pandas dataframe
    df = pd.DataFrame(arr)

    header = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=header)

    return df

In [ ]:
mod_df_Nd['NDVI'].plot()